In [12]:
import numpy as np
from scipy.sparse import diags, kron, csr_matrix, csc_matrix
from scipy.sparse.linalg import spsolve, norm, inv, eigsh
import matplotlib.pyplot as plt
from matplotlib import rc
import scipy as sp
rc('text', usetex=True) # para usar latex en matplotlib

*Universidad de Chile*  
*Facultad de Ciencias Físicas y Matemáticas*  
*Departamento de Ingeniería Matemática*

**MA5307-1 Análisis Numérico de EDP: Teoría y Laboratorio**  
**Profesor:** Axel Osses  
**Auxiliares:** Emir Chacra y Vicente Salinas

**Integrantes:** Sebastián Cobaise, Arturo Lazcano, Benjamin Tardy

# Laboratorio 1

El objetivo de esta sesión práctica es aprender a resolver EDP’s en 2
dimensiones mediante el Método de las Diferencias Finitas.

Específicamente se resolverá la ecuación de Poisson en un rectángulo unitario y en un dominio perforado, además se estudiará la aproximación para condiciones de borde en dominios con curvatura

# Parte a


## Ecuación de Poisson

Considere la ecuación dada por

$$
\begin{equation}
\left.
\begin{array}[c]{rll}
-\Delta u= & 0 & \text{sobre }\Omega=\left[  0,1\right]^{2}\\
u(0,y)=u(1,y)= & 0 & \text{en }0\leq y\leq 1\\
u(x,0)= & 0 & \text{en }0\leq x\leq 1\\
u(x,1)= & g(x) & \text{en }0\leq x\leq 1
\end{array}
\right\} \quad (1)
\end{equation}
$$

Sea $N\in\mathbb{N}$, considere la aproximación del cuadrado unitario dado por la malla de puntos
$$
\Omega_{h}=\left\{  \left(  x_{j},y_{k}\right)  \mid j,k\in\left\{
0,1,\ldots,N+1\right\}  \right\}
$$
donde $x_{j}=jh$, $y_{k}=kh$, $h=\dfrac{1}{N+1}$. Se define el operador laplaciano discretizado por 5 puntos como
$$
\Delta_{N}u_{j,k}=\dfrac{1}{h^{2}}\left(  u_{j+1,k}+u_{j-1,k}
+  u_{j,k-1}+u_{j,k+1}
-4u_{jk}\right)
$$
donde $u_{jk}$ aproxima a $u\left(x_{j},y_{k}\right)  $.

Tomando $g(x) = \sin(\pi x)$, se puede demostrar que la solución única de esta ecuación está dada por

$$u\left(  x,y\right)  =\frac{\sin\left( \pi x\right)   \sinh\left( \pi y\right)  }{\sinh\left(  \pi\right) } $$

**P1.** Escriba dos funciones que calculen $\boldsymbol{A}_{h}$ y $\boldsymbol{b}_{h}$ de la forma más simple posible. Las entradas para estas funciones deben ser $N$ y $g$.

También escriba una función `solve` que, utilizando estas funciones (y las funciones auxiliares que necesite), entregue la solución aproximada.

**Indicación.** Revise la documentación del comando `kron` (disponible a través de `scypy.sparse`)

In [30]:
def Ah(N): 
  h = 1/(N+1) 
  e = np.ones(N) #Este comando define un vector de largo n lleno de 1's. Existe también np.zeros. 
  f = np.ones(N-1) 
  k = np.array([-f,4*e,-f], dtype= object) #Lista con los vectores 
  offset = [-1,0,1] #Posiciones respecto a la diagonal en que se ubicarán los vectores 
  B = sp.sparse.diags(k,offset) #Definir matriz sparse diagonal 
  print(B.shape)
  I = np.eye(N) 
  A = np.kron(I,B)
  h1 = np.ones(N**2-N)
  k1 = np.array([-h1,-h1])
  offset1 = [-N,N]
  diag=sp.sparse.diags(k1,offset1)
  print(A.shape)
  print(diag.shape)
  return ((1/h)**2)*(A+diag) 
 
def bh(N,f,g): 
  h = 1/(N+1) 
  b = np.zeros(N**2)
  for j in range(1,N+1):
    for i in range(1,N+1):
      b[(j-1)*N+i-1]=f(i*h,j*h) 
      if j==N:
        b[(j-1)*N+i-1]+=(g(i*h)/h**2)
  return b
def solve(N, f, g):
    """Resuelve el problema del laplaciano en 2D en (0,1)^2, con condiciones
    Dirichlet en el borde, usando una grilla uniforme, con tamano de
    paso h=1/N en x e y.
    """
    sol = sp.sparse.linalg.spsolve(Ah(N),bh(N,f,g))
    sol1= np.reshape(sol, (N,N))
    return sol1

**P2.** Resuelva la ecuación para $N = 10, 20, 30, 40, 50$. Comente sus resultados.

In [31]:
#El error econtrado en el lab es que hay una diferencia en la dimensión de las matrices, problablemente haya una dimensión mal definida (N en vez de N**2)
N=20
def f1(x,y):
  return 0
def g1(x):
  return np.sin(np.pi*x) 
plt.imshow(solve(N,f1,g1))
plt.colorbar()
plt.show()

(20, 20)
(20, 20)
(400, 400)


ValueError: ignored

**P3.** Estudie el condicionamiento en norma $2$ de la Matriz $A_h$.

**Indicación.** Sean $p\in\left[  1,+\infty\right]  $ y $\boldsymbol{A}\in\mathbb{R}^{N\times N}$. Se define la norma inducida $p$ de $\boldsymbol{A}$ como
$$
\left\Vert \boldsymbol{A}\right\Vert _{p}=\sup\limits_{\boldsymbol{x\in}\mathbb{R}^{N}\setminus\left\{  \boldsymbol{0}\right\}  }\dfrac{\left\Vert\boldsymbol{Ax}\right\Vert _{p}}{\left\Vert \boldsymbol{x}\right\Vert_{p}}
$$
y, si $\boldsymbol{A}$ es invertible, el número de condición en norma inducida $p$ como $\operatorname{cond}_{p}\left(  \boldsymbol{A}\right)  =\left\Vert
\boldsymbol{A}\right\Vert _{p}\left\Vert \boldsymbol{A}^{-1}\right\Vert _{p}$.

En este caso particular, como $\boldsymbol{A}_{h}$ es simétrica, el número de condición en la norma $2$ inducida puede calcularse como
$$
\operatorname{cond}_{2}\left(  \boldsymbol{A}_{h}\right)  =\dfrac{\lambda_{\max,h}\left(  \boldsymbol{A}_{h}\right)  }{\lambda_{\min,h}\left(\boldsymbol{A}_{h}\right)}
$$
donde $\lambda_{\max,h}\left(  \boldsymbol{A}_{h}\right)  =\max\left\{\left\vert \lambda\right\vert \mid\lambda\in\sigma\left(  \boldsymbol{A}_{h}\right)  \right\}  $ y $\lambda_{\min,h}\left(  \boldsymbol{A}_{h}\right)
=\min\left\{  \left\vert \lambda\right\vert \mid\lambda\in\sigma\left(\boldsymbol{A}_{h}\right)  \right\}  $.

In [ ]:
def cond_2_sparse(A):
    """
    Calcula el numero de condicionamiento en norma inducida 2
    para una matriz sparse simetrica A.
    Se obtiene como el valor propio de mayor módulo dividido por el de menor módulo
    
    Input:
    - A matriz sparse simetrica
    
    Documentacion de eigsh:
    https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.sparse.linalg.eigsh.html
    """

    
    
    return cond

# Parte b

## Ecuación en Dominios Perforados

Para $\Omega = [0, 1]^2\backslash B ((0.5, 0.5), 0.3)$, considere:
$$
\left(  P \right)  \left\{
\begin{array}
[c]{ccc}
-\Delta u(x,y)=&0,  & \text{si }(x,y)\text{ en }\Omega \\
u(x,y)=&g(x,y), & \text{si} (x,y)\text{ en }\partial\Omega
\end{array}
\right.
$$

y $g$ dada por:

$$
g\left(  x,y\right)  =\left\{
\begin{array}
[c]{cc}
2  & \text{si }(x,y) \in \partial [0,1]^2 \\
0  & \text{si }(x,y) \in \partial B ((0.5, 0.5), 0.3)
\end{array}
\right.
$$






**P1.** Genere una función que reciba un punto de la malla $(x_j , y_k)$ y entregue como resultado $1$ si es un
punto interior al cual se le puede calcular $\Delta u$ con la aproximación utilizada en la fórmula de $5$ puntos.

In [9]:
def xx(xj,yk,h):
  if xj+h<=1 and (xj+h-0.5)**2+(yk-0.5)**2>0.3**2 and (xj+h)>=0:
    if xj-h<=1  and (xj-h-0.5)**2+(yk-0.5)**2>0.3**2 and (xj-h)>=0:
      return 1
  else:
    return 0

def yy(xj,yk,h):
  if yk+h<=1 and (xj-0.5)**2+(yk+h-0.5)**2>0.3**2 and yk+h>0:
    if yk-h<=1 and (xj-0.5)**2+(yk-h-0.5)**2>0.3**2 and yk-h>0:
      return 1
  else:
    return 0

def interior(xj,yk,h):
  return xx(xj,yk,h)*yy(xj,yk,h)

**P2.** Asuma que la matriz del sistema es de la forma $A = A_x + A_y$, donde $A_x$ tiene los coeficientes adecuados
para la aproximación de la derivada parcial según $x$ y lo equivalente para $A_y$. Encuentre la forma
que deben tener $A_x$ y $A_y$ y escriba un programa que calcule dichas matrices.

In [8]:
def Ahx(N):
  h=1/(N+1)
  Ax=np.zeros((N**2,N**2))
  for j in range(1,N+1):
    for i in range(1,N+1):
      if xx(i*h,j*h,h)==1:
        row= np.zeros(N**2)
        row[(j-1)*N+i-1]=2
        if i==1:
          row[(j-1)*N+i+1-1]=-1
        if i==N:
          row[(j-1)*N+i-1-1]=-1
        else:
          row[(j-1)*N+i+1-1]=-1
          row[(j-1)*N+i-1-1]=-1
      else:
        if (i*h)**2+(j*h)**2<0.3**2:
          row= np.zeros(N**2)
          row[(j-1)*N+i-1]=1
        else:

          if i*h<0.5:
            hbarra=abs((0.5-np.sqrt(0.3**2-(j*h-0.5)**2))-i*h)
            gamma=2/(hbarra*h+hbarra**2)
            alfa=2/(hbarra*h+h**2)
            beta=-alfa-gamma          
            row= np.zeros(N**2)
            row[(j-1)*N+i-1]=beta
            row[(j-1)*N+i-1-1]=alfa

          else: #i*h>0.5
            hbarra=abs(i*h-(0.5+np.sqrt(0.3**2-(j*h-0.5)**2)))
            gamma=2/(hbarra*h+hbarra**2)
            alfa=2/(hbarra*h+h**2)
            beta=-alfa-gamma          
            row= np.zeros(N**2)
            row[(j-1)*N+i-1]=beta
            row[(j-1)*N+i+1-1]=alfa
      Ax[(j-1)*N+i-1,:]=row
  return ((1/h)**2)*Ax

def Ahy(N):
  h=1/(N+1)
  Ay=np.zeros((N**2,N**2))
  for j in range(1,N+1):
    for i in range(1,N+1):
      if yy(i*h,j*h,h)==1:
        row= np.zeros(N**2)
        row[(j-1)*N+i-1]=2
        if j==1:
          row[(j-1)*N+i+N-1]=-1
        if j==N:
          row[(j-1)*N+i-N-1]=-1
        else:
          row[(j-1)*N+i+N-1]=-1
          row[(j-1)*N+i-N-1]=-1
      else:
        if (i*h)**2+(j*h)**2<0.3**2:
          row= np.zeros(N**2)
          row[(j-1)*N+i-1]=1
        else:

          if j*h<0.5:
            hbarra=abs((0.5-np.sqrt(0.3**2-(i*h-0.5)**2))-j*h)
            gamma=2/(hbarra*h+hbarra**2)
            alfa=2/(hbarra*h+h**2)
            beta=-alfa-gamma          
            row= np.zeros(N**2)
            row[(j-1)*N+i-1]=beta
            row[(j-1)*N+i-N-1]=alfa

          else: #i*h>0.5
            hbarra=abs((0.5-np.sqrt(0.3**2-(i*h-0.5)**2))-j*h)
            gamma=2/(hbarra*h+hbarra**2)
            alfa=2/(hbarra*h+h**2)
            beta=-alfa-gamma          
            row= np.zeros(N**2)
            row[(j-1)*N+i-1]=beta
            row[(j-1)*N+i+N-1]=alfa
      Ay[(j-1)*N+i-1,:]=row
  
  return ((1/h)**2)*Ay

**P3.** Escriba una función que calcule $b_h$ adaptado a este caso.

In [7]:
def bh(N):
  h=1/(N+1)
  b=np.zeros(N**2)
  for j in range(1,N+1):
    for i in range(1,N+1):
      if i==1:
        b[(j-1)*N+i-1]+=(2/h**2)
      if i==N:
        b[(j-1)*N+i-1]+=(2/h**2)
      if j==1:
        b[(j-1)*N+i-1]+=(2/h**2)
      if j==N:
        b[(j-1)*N+i-1]+=(2/h**2)
  return b

def solve2(N):
  Ax=Ahx(N)
  Ay=Ahy(N)
  Ah=Ax+Ay
  b=bh(N)
  sol = sp.sparse.linalg.spsolve(Ah,b)
  return sol

**P4.** Resuelva el sistema y grafique la solución aproximada para $N = 10, 20, 30, 40, 50$. Comente sus resultados.

**P5.** ¿Qué sucede con el condicionamiento de la matriz utilizando la fórmula de la Parte a?¿Cómo adaptaría su código para el caso $f\neq 0$?